Analysis plan 
1. convert fMRI to fsaverage space 
2. extract time series from seeds 
3. correlate seed time series with rest of the brain 
4. convert to fisher-z
5. write a correlation image

6. correlate seed time series with time series from glasser atlas
7. convert to fisher-z scores

8. test seed correlation differences between 2 conditions on voxels & parcels
9. find peaks in the prefrontal cortex  


In [1]:
import os 
import numpy as np
import pandas as pd
import subprocess
import nibabel as nb
from nibabel.freesurfer.io import read_label

In [2]:
def define_data():
    # read label
    BASE_PATH = os.path.dirname(os.getcwd())
    label = read_label(BASE_PATH + '/seeds/TMS10mm-lh.label')

    # get fMRI 
    FMRI_PATH = '/Users/laurituominen/Documents/Research/FDGPET/petanalysis/test_fMRI/'
    f = 'sub-tbsfdg001_ses-002_task-rest_space-MNI152NLin6Asym_res-2_desc-denoised_bold.nii.gz'
    fMRI_file = FMRI_PATH + f
    
    # get parcel timeseries 
    ts = 'sub-tbsfdg001_ses-002_task-rest_space-MNI152NLin6Asym_atlas-Glasser_timeseries.tsv'
    parcel_file = FMRI_PATH + ts
    return((label, fMRI_file, parcel_file))


In [3]:
def surf2vol(fMRI_file):
    FMRI_PATH = os.path.dirname(fMRI_file)
    split_id = os.path.basename(fMRI_file).split('_')
    id = split_id[0] + '_' + split_id[1]
    surface = os.path.join(FMRI_PATH, id + '.lh.fsaverage.preprocessed.nii.gz')

    # vol2surf 
    cmd = [
    "mri_vol2surf",
    "--mov", fMRI_file,
    "--mni152reg",
    "--hemi", "lh",
    "--surf", "white",
    "--o", surface
    ]
    # Run the command
    result = subprocess.run(cmd, capture_output=True, text=True)
    return(surface)    

In [4]:
def get_awf(label, surface):
    # extract seed 
    img_L = nb.load(surface).get_fdata()
    img_L = np.squeeze(img_L)
    awf = np.mean(img_L[label,:],0)

    # save awf
    split_id = os.path.basename(surface).split('.')
    id = split_id[0]
    FMRI_PATH = os.path.dirname(surface)
    np.savetxt(os.path.join(FMRI_PATH, id + '_awf.txt'), awf)
    
    return(awf)

In [5]:
def seed2vox(full_file, awf):

    # load fMRI 
    fMRI = nb.load(full_file)
    fMRI_data =fMRI.get_fdata()
    
    # Calculate means, std, and normalize
    fMRI_mean = np.mean(fMRI_data, axis=3, keepdims=True)
    fMRI_std = np.std(fMRI_data, axis=3, keepdims=True)
    epsilon = 1e-8
    fMRI_std_adjusted = fMRI_std + epsilon
    normalized_fMRI = (fMRI_data - fMRI_mean) / fMRI_std_adjusted
    
    # variance normalize seed time series
    awf_mean = np.mean(awf)
    awf_std = np.std(awf)
    normalized_seed = (awf - awf_mean) / awf_std
   
    # calculate correlation between seed and voxels
    seed_correlations = np.dot(normalized_fMRI, normalized_seed) / normalized_seed.shape[0]
    zscored = np.arctanh(seed_correlations)
    
    # save data 
    corr_img = nb.Nifti1Image(seed_correlations, fMRI.affine, nb.Nifti1Header())
    zscored_img = nb.Nifti1Image(zscored, fMRI.affine, nb.Nifti1Header())

    # write data 
    split_id = os.path.basename(full_file).split('_')
    id = split_id[0] + '_' + split_id[1]
    FMRI_PATH = os.path.dirname(full_file)
    nb.save(corr_img, os.path.join(FMRI_PATH, id + '.seed_to_voxel_correlations.nii'))
    nb.save(zscored_img, os.path.join(FMRI_PATH, id + '.seed_to_voxel_zscore.nii'))

    # return        
    return((corr_img, zscored_img))

In [ ]:
# run analysis 
label, fMRI_file, parcel_file = define_data()

surface = surf2vol(fMRI_file)

awf = get_awf(label,surface)

corr_img, zscored_img = seed2vox(fMRI_file, awf)

